In [1]:
import numpy as np
import numpy as np
import pandas as pd
#import WindFarmGenetic_v1 # wind farm layout optimization using genetic algorithms classes
from datetime import datetime
import os
from sklearn.svm import SVR
import pickle


In [2]:
import os
os.getcwd()

'/home/naveen/arena/hackathon/shell/hackathon/python/Ver1_N'

In [3]:
##
wt_N=50  #number of turbines
NA_loc=[]  #index of obstructed locations
population_size=10  #population size/number of layouts for genetic algorithms
n_mc_samples=100 #population/layout for SVM training
iteration_times=100  #number of iterations

# wind farm size, cells
cols_cells = 80  # number of cells each row
rows_cells = 80  # number of cells each column
cell_width = 50 # unit (TBD)--> have to be computed.
constraint_dist = 400 #unit distance between two turbines
constraint_perim = 50

radius=np.ceil(constraint_dist/cell_width)

#genetic algorithmic parameters.
elite_rate = 0.2 
cross_rate = 0.6
mutate_rate = 0.1

random_rate = 0.5
num_runs=10 #number of independent runs with random initialization.


In [4]:
perim_cells=np.ceil(constraint_perim/cell_width) #TBU generalize it for multiple boundary cells.
NA_loc=[] 
NA_loc.extend([i for i in range(cols_cells)]) #upper row
NA_loc.extend([i*(cols_cells) for i in range(rows_cells)]) #left boundary
NA_loc.extend([i*(cols_cells)+cols_cells-1 for i in range(rows_cells)])#right Boundary
NA_loc.extend([(cols_cells)*(rows_cells-1)+i for i in range(rows_cells)])#right Boundary


In [5]:
data_folder = "naveen_data" #temporary code related files. (MOdify it as per your specification)
svr_model_filename = 'svr_1s1d_N_13.svr' #svr training file.
if not os.path.exists(data_folder):
    os.makedirs(data_folder)

wds_data_folder = "{}/wds".format(data_folder)
if not os.path.exists(wds_data_folder):
    os.makedirs(wds_data_folder)
    
# initial populations saved folder
init_pops_data_folder = "{}/init_data".format(data_folder)
if not os.path.exists(init_pops_data_folder):
    os.makedirs(init_pops_data_folder)

#result folder
results_data_folder = data_folder+"/results"
if not os.path.exists(results_data_folder):
    os.makedirs(results_data_folder)
    
#result folder for sugga genetic algorithm
sg_result_folder = "{}/sg".format(results_data_folder)
if not os.path.exists(sg_result_folder):
    os.makedirs(sg_result_folder)

In [6]:
import WindFarmGenetic_v1 as WindFarmGenetic
wfg = WindFarmGenetic.WindFarmGenetic(rows=rows_cells, cols=cols_cells, N=wt_N, NA_loc=NA_loc, pop_size=population_size,
                                      iteration=iteration_times,cell_width=cell_width, elite_rate=elite_rate,
                                             cross_rate=cross_rate, random_rate=random_rate, mutate_rate=mutate_rate,constraint_dist=constraint_dist)

In [7]:
for i in range(num_runs):
    wfg.gen_init_pop_NA()
    wfg.save_init_pop_NA("{}/init_{}.dat".format(init_pops_data_folder, i),
                         "{}/init_{}_NA.dat".format(init_pops_data_folder, i))

Generation layout: 100%|██████████| 10/10 [00:03<00:00,  2.78it/s]


In [8]:
##Generate initial layouts: single layout--> 1d array of size row*col, layouts are set of num_training_sample_SVM different(random) layouts.  
train_mc_layouts, train_mc_layouts_NA = WindFarmGenetic.LayoutGridMCGenerator.gen_mc_grid_with_NA_loc_circle(rows_cells,
                                                                                                      cols_cells,
                                                                                                      n_mc_samples,
                                                                                                      wt_N, NA_loc,radius,
                                                                                                      "{}/mc_layout.dat".format(wds_data_folder),
                                                                                                      "{}/mc_layout_NA.dat".format(wds_data_folder))

layouts = np.genfromtxt("{}/mc_layout.dat".format(wds_data_folder), delimiter="  ", dtype=np.int32)

Generation layout: 100%|██████████| 100/100 [00:35<00:00,  2.81it/s]


In [9]:

wfg.mc_gen_xy_NA_hackathon(rows=rows_cells, cols=cols_cells, layouts=layouts, n=n_mc_samples, N=wt_N,  xfname="{}/x.dat".format(wds_data_folder),
                 yfname="{}/y.dat".format(wds_data_folder))

layouts: 100%|██████████| 100/100 [00:23<00:00,  4.30it/s]


In [10]:
x_original = pd.read_csv("{}/x.dat".format(wds_data_folder), header=None, nrows=rows_cells * cols_cells,
                         delim_whitespace=True, dtype=np.float32)
x_original = x_original.values

In [11]:
y_original = pd.read_csv("{}/y.dat".format(wds_data_folder), header=None, nrows=rows_cells * cols_cells,
                         delim_whitespace=True, dtype=np.float32)
y_original = y_original.values.flatten()

In [12]:
print(x_original.shape)
print(y_original.shape)

(6400, 2)
(6400,)


In [ ]:
# create a SVR object and specify the kernal and other parameters
svr_model = SVR(kernel='rbf', C=2000.0, gamma=0.3, epsilon=.1)
# build the SVR power distribution model
svr_model.fit(x_original, y_original)

# save the SVR model to a file
pickle.dump(svr_model, open("{}/{}".format(wds_data_folder, svr_model_filename), 'wb'))

In [ ]:
del x_original
del y_original

In [7]:
svr_model=pickle.load(open("{}/{}".format(wds_data_folder, svr_model_filename), 'rb'))

In [8]:
##### SUGGA: support vector regression guided genetic algorithm
result_arr = np.zeros((num_runs, 2), dtype=np.float32)
for i in range(0, num_runs):  # run times
    print("run number: {} ...".format(i))
    wfg.load_init_pop_NA("{}/init_{}.dat".format(init_pops_data_folder, i),
                         "{}/init_{}_NA.dat".format(init_pops_data_folder, i))
    run_time, eta = wfg.sugga_genetic_alg(i, svr_model=svr_model, result_folder=sg_result_folder)
    result_arr[i, 0] = run_time
    result_arr[i, 1] = eta
time_stamp = datetime.now().strftime("%Y%m%d%H%M%S")
filename = "{}/result_sugga_{}.dat".format(sg_result_folder, time_stamp)
np.savetxt(filename, result_arr, fmt='%f', delimiter="  ")

run number: 0 ...


layouts:   0%|          | 0/10 [00:00<?, ?it/s]

Support vector regression guided genetic algorithm starts....
generation 0...


layouts: 100%|██████████| 10/10 [00:02<00:00,  4.64it/s]


[511.57404 513.1111  513.5966  513.7728  510.86108 513.0939  511.29047
 513.6769  511.7074  510.65308]
3
corss_point:1
size: m_ov-->47 m_nov-->3 f_ov-->46 f_nov-->4
cross_point: f_nov-->1 m_nov-->2
cross_point: f_nov-->[2638] m_nov-->[5967 2243]
50
size: m_ov-->47 m_nov-->3 f_ov-->46 f_nov-->4
cross_point: m_nov-->1 f_nov-->3
cross_point: m_nov-->[3707] f_nov-->[2415 4123 2186]
50
2
corss_point:1
size: m_ov-->47 m_nov-->3 f_ov-->48 f_nov-->2
cross_point: f_nov-->1 m_nov-->2
cross_point: f_nov-->[350] m_nov-->[2682 3015]
50
size: m_ov-->47 m_nov-->3 f_ov-->48 f_nov-->2
cross_point: m_nov-->1 f_nov-->1
cross_point: m_nov-->[1043] f_nov-->[971]
50
3
corss_point:2
size: m_ov-->46 m_nov-->4 f_ov-->47 f_nov-->3
cross_point: f_nov-->2 m_nov-->2
cross_point: f_nov-->[3707 5967] m_nov-->[2186 2638]
50
size: m_ov-->46 m_nov-->4 f_ov-->47 f_nov-->3
cross_point: m_nov-->2 f_nov-->1
cross_point: m_nov-->[2415 4123] f_nov-->[2243]
50
2
corss_point:1
size: m_ov-->48 m_nov-->2 f_ov-->47 f_nov-->3
cros

...

layouts:  90%|█████████ | 9/10 [00:01<00:00,  4.68it/s]

Somewhere proximity constraint is violated



layouts: 100%|██████████| 10/10 [00:02<00:00,  4.54it/s]


[512.43463 507.1625  512.2505  514.1868  508.2904  511.6118  511.52817
 512.1599  512.6181  514.40924]
2
corss_point:1
size: m_ov-->47 m_nov-->3 f_ov-->48 f_nov-->2
cross_point: f_nov-->1 m_nov-->2
cross_point: f_nov-->[6208] m_nov-->[ 174 4435]
50
size: m_ov-->47 m_nov-->3 f_ov-->48 f_nov-->2
cross_point: m_nov-->1 f_nov-->1
cross_point: m_nov-->[3711] f_nov-->[1347]
50
2
corss_point:1
size: m_ov-->48 m_nov-->2 f_ov-->47 f_nov-->3
cross_point: f_nov-->1 m_nov-->1
cross_point: f_nov-->[2638] m_nov-->[3707]
50
size: m_ov-->48 m_nov-->2 f_ov-->47 f_nov-->3
cross_point: m_nov-->1 f_nov-->2
cross_point: m_nov-->[2586] f_nov-->[1043 2586]
49


ValueError: could not broadcast input array from shape (49) into shape (50)

In [8]:
np.random.randint(1, 1)

ValueError: low >= high

In [9]:
l=np.linspace(0,10,20)
l

array([ 0.        ,  0.52631579,  1.05263158,  1.57894737,  2.10526316,
        2.63157895,  3.15789474,  3.68421053,  4.21052632,  4.73684211,
        5.26315789,  5.78947368,  6.31578947,  6.84210526,  7.36842105,
        7.89473684,  8.42105263,  8.94736842,  9.47368421, 10.        ])

In [11]:
l[1:]

array([ 0.52631579,  1.05263158,  1.57894737,  2.10526316,  2.63157895,
        3.15789474,  3.68421053,  4.21052632,  4.73684211,  5.26315789,
        5.78947368,  6.31578947,  6.84210526,  7.36842105,  7.89473684,
        8.42105263,  8.94736842,  9.47368421, 10.        ])

### Shell Power Computation Package

In [1]:
import sys 
sys.path.append('../Shell_Code_Modified/')

In [2]:
import Farm_Evaluator_Vec

In [3]:
total_energy, turbine_power=Farm_Evaluator_Vec.get_AEP_results()

In [4]:
total_energy

array([1.2523587, 1.1595447, 1.1450489, 1.0768385, 1.176319 , 1.1840583,
       1.13568  , 1.079648 , 1.0935222, 1.1273198, 1.0859367, 1.167106 ,
       1.1016386, 1.124391 , 1.1225449, 1.100678 , 1.1647563, 1.2282459,
       1.1581823, 1.197542 , 1.2453799, 1.2442212, 1.2059841, 1.1259376,
       1.2380294, 1.1106503, 1.1325871, 1.1829029, 1.1153044, 1.1439447,
       1.1912581, 1.1264657, 1.1022447, 1.1789318, 1.1049944, 1.2037905,
       1.220672 , 1.1373823, 1.2131999, 1.1142861, 1.199062 , 1.141587 ,
       1.1335138, 1.1493688, 1.0722657, 1.0965657, 1.2305766, 1.2662541,
       1.1084578, 1.0826805], dtype=float32)

In [5]:
turbine_power

505.4506365966797